In [82]:
train_file=r'D:\Programming\PROJECTS\Data Science\FSDS\Californa_Housing-ML_Pipeline\housing\artifact\data_ingestion\2022-08-10T00-50-00\ingested_data\train\housing.csv'

In [83]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')

In [84]:
df=pd.read_csv(train_file)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN


In [85]:
df.shape

(16512, 10)

In [86]:
x,y=df.drop(['median_house_value'],axis=1),df[['median_house_value']]

In [87]:
x.shape, y.shape

((16512, 9), (16512, 1))

In [88]:
x.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [89]:
num_col=x.select_dtypes(exclude='O')
num_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16354 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
dtypes: float64(8)
memory usage: 1.0 MB


In [90]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='median')
imputer.fit(num_col)

SimpleImputer(strategy='median')

In [91]:
imputer.feature_names_in_

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype=object)

In [92]:
imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [93]:
num_col.median()

longitude             -118.51000
latitude                34.26000
housing_median_age      29.00000
total_rooms           2119.00000
total_bedrooms         433.00000
population            1164.00000
households             408.00000
median_income            3.54155
dtype: float64

In [94]:
imputer.n_features_in_

8

In [95]:
num_col_transformed=imputer.transform(num_col)

In [96]:
cat_col=x.select_dtypes('O')
cat_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ocean_proximity  16512 non-null  object
dtypes: object(1)
memory usage: 129.1+ KB


In [97]:
cat_imputer=SimpleImputer(strategy='most_frequent')
cat_col_transformed=cat_imputer.fit_transform(cat_col)

In [98]:
cat_imputer.feature_names_in_

array(['ocean_proximity'], dtype=object)

In [99]:
cat_imputer.statistics_

array(['<1H OCEAN'], dtype=object)

In [100]:
cat_col.mode()

,ocean_proximity
0,<1H OCEAN


In [101]:

# * Column Name of Data
COLUMN_LONGITUDE = "longitude"
COLUMN_LATITUDE = "latitude"
COLUMN_HOUSING_MEDIAN_AGE = "housing_median_age"
COLUMN_TOTAL_ROOMS = "total_rooms"
COLUMN_TOTAL_BEDROOMS = "total_bedrooms"
COLUMN_POPULATION = "population"
COLUMN_HOUSEHOLDS = "households"
COLUMN_MEDIAN_INCOME = "median_income"
COLUMN_MEDIAN_HOUSE_VALUE = "median_house_value"
COLUMN_OCEAN_PROXIMITY = "ocean_proximity"
TARGET_COLUMN = "median_house_value"

class FeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        add_bedrooms_per_room: bool = True,
        total_rooms_idx: int = 3,
        total_bedrooms_idx: int = 4,
        population_idx: int = 5,
        households_idx: int = 6,
        columns: list = None,
    ):
        """Generate Additional Feature for prediction

        Args:
            add_bedrooms_per_room (bool, optional): To add bedrooms_per_room feature. Defaults to True.
            total_rooms_idx (int, optional): index of total_rooms. Defaults to 3.
            total_bedrooms_idx (int, optional): index of total_bedrooms. Defaults to 4.
            population_idx (int, optional): index of population. Defaults to 5.
            households_idx (int, optional): index of households. Defaults to 6.
            columns (list, optional): list of columns of dataframe in same sequence to get index of feature. Defaults to None.
        """
        try:
            self.columns = columns
            if self.columns is not None:
                total_rooms_idx = self.columns.index(COLUMN_TOTAL_ROOMS)
                population_idx = self.columns.index(COLUMN_POPULATION)
                households_idx = self.columns.index(COLUMN_HOUSEHOLDS)
                total_bedrooms_idx = self.columns.index(COLUMN_TOTAL_BEDROOMS)

            self.add_bedrooms_per_room = add_bedrooms_per_room
            self.total_rooms_idx = total_rooms_idx
            self.population_idx = population_idx
            self.households_idx = households_idx
            self.total_bedrooms_idx = total_bedrooms_idx
        except Exception as e:
            raise Exception(e) from e

    def fit(self, X, y=None):
        return self

    def transform(self, X: np.ndarray, y: np.ndarray = None) -> np.ndarray:
        """Add additional feature at last position of columns in the following sequence:
            Data, room_per_household, population_per_household, bedrooms_per_room

        Args:
            X (np.ndarray): Feature Data
            y (np.ndarray, optional): Target. Defaults to None.

        Raises:
            HousingException: Exception

        Returns:
            np.ndarray: return data with additional feature
        """
        try:
            room_per_household = X[:, self.total_rooms_idx] / X[:, self.households_idx]
            population_per_household = X[:, self.population_idx] / X[:, self.households_idx]
            if self.add_bedrooms_per_room:
                bedrooms_per_room = X[:, self.total_bedrooms_idx] / X[:, self.total_rooms_idx]
                generated_feature = np.c_[X, room_per_household, population_per_household, bedrooms_per_room]
            else:
                # or np.hstack
                generated_feature = np.c_[X, room_per_household, population_per_household]
            return generated_feature
        except Exception as e:
            raise Exception(e) from e


In [102]:
add_feature=FeatureGenerator()

In [103]:
x.iloc[0]

longitude            -121.46
latitude               38.52
housing_median_age      29.0
total_rooms           3873.0
total_bedrooms         797.0
population            2237.0
households             706.0
median_income         2.1736
ocean_proximity       INLAND
Name: 0, dtype: object

In [104]:
add_feature.transform(x.values)[0]

array([-121.46, 38.52, 29.0, 3873.0, 797.0, 2237.0, 706.0, 2.1736,
       'INLAND', 5.485835694050992, 3.168555240793201,
       0.20578363026077975], dtype=object)

In [105]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [106]:
num_pipe=Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('feature_gen',FeatureGenerator()),('scaler',StandardScaler())])
cat_pipe=Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),('ohe',OneHotEncoder()),('scaler',StandardScaler(with_mean=False))]) # with_mean=False is due to sparse matrix from OneHotEncoder

In [107]:
preprocessing_obj = ColumnTransformer(
    transformers=[("num_pipe", num_pipe, num_col.columns.tolist()), ("cat_pipe", cat_pipe, cat_col.columns.tolist())]
)


In [108]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
16507,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,268500.0,<1H OCEAN
16508,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,90400.0,INLAND
16509,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,140400.0,<1H OCEAN
16510,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,258100.0,<1H OCEAN


In [109]:
x_transformed=preprocessing_obj.fit_transform(x)
x_transformed

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.        ,
         0.        ,  2.9869105 ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ...,  0.        ,
         0.        ,  0.        ],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.        ,
         0.        ,  0.        ]])

In [110]:
from sklearn.linear_model import LinearRegression

lr=LinearRegression()

In [111]:
lr.fit(x_transformed,y)
lr.score(x_transformed,y)

0.648155362842701

In [112]:
lr.coef_

array([[-5.56496734e+04, -5.67116339e+04,  1.37347208e+04,
        -1.94305839e+03,  7.34324001e+03, -4.57092842e+04,
         4.54532604e+04,  7.47141639e+04,  6.60457916e+03,
         1.04306233e+03,  9.24831738e+03, -1.79278548e+13,
        -1.68261738e+13, -3.97394837e+11, -1.13817271e+13,
        -1.20895785e+13]])

In [124]:
a=None

In [125]:
if a:
    print('a')
else:
    a='asd'

if a:
    print(a)

asd


In [142]:
import pandas as pd
df=pd.DataFrame({'a':[*'12345678a90']})
df

,a
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,a
9,9


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a       11 non-null     object
dtypes: object(1)
memory usage: 216.0+ bytes


In [144]:
df['a'].astype(float)

ValueError: could not convert string to float: 'a'

In [145]:
df

,a
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,a
9,9


In [138]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a       10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes
